In [3]:
import sys
sys.path.append("..")

In [4]:
from at_detector import ATDetector
from constants import Config, Topics
import cv2
from cortano import RemoteInterface
from robot_interface_node import ManualControl
import pygame
import numpy as np

pygame 2.5.1 (SDL 2.28.2, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
def enoughDataRecursive(foundData, inputTags, start_tag):
    # print(foundData, inputTags, start_tag)
    for tagSet in inputTags:
        if start_tag in tagSet:
            for tag in tagSet:
                foundData[tag] = True
                copyInputTags = inputTags.copy()
                copyInputTags.remove(tagSet)
                enoughDataRecursive(foundData, copyInputTags, tag)

def enoughData(detectTagSets, allTags):
    inputTags = [tagSet for tagSet in detectTagSets if len(tagSet) > 1]
    if len(inputTags) == 0:
        return False
    foundData = {tag: False for tag in allTags}
    currentSet = inputTags[0]
    for tag in currentSet:
        foundData[tag] = True
        if len(inputTags) > 1:
            enoughDataRecursive(foundData, inputTags[1:], tag)
    return all(foundData.values())

# tests
setOfTags = {1,2,3,4}
detectedTagIds = [(1,), (2,3,4)]
assert(enoughData(detectedTagIds, setOfTags) == False)
detectedTagIds = [(1,3), (2,3,4)]
assert(enoughData(detectedTagIds, setOfTags))
detectedTagIds = [(1,3), (2,4)]
assert(enoughData(detectedTagIds, setOfTags) == False)
detectedTagIds = [(1,3), (2,3,4)]*4
assert(enoughData(detectedTagIds, setOfTags))
detectedTagIds = [(1,3), (2,4)]*4
assert(enoughData(detectedTagIds, setOfTags) == False)
detectedTagIds = [(3, 4), (1, 3), (1, 2)]
assert(enoughData(detectedTagIds, setOfTags))

In [ ]:
setOfTags = {1,2,3,4}
detectTagSets = []
setOfDetectedTags = set()
usefulTags = []

robot = RemoteInterface(Config.ip)
at_detector = ATDetector()
manualControl = ManualControl(robot)

while not enoughData(detectTagSets, setOfTags):
    robot.update()
    manualControl.run()
    color, _, _ = robot.read()
    tags = at_detector.detect(color)
    tags = [tag for tag in tags if tag.tag_id in setOfTags]
    if len(tags) < 2:
        continue
    tagIds = tuple([tag.tag_id for tag in tags])
    print("detected ids:", tagIds)
    if not tagIds in setOfDetectedTags:
        print("new set:", tagIds)
        setOfDetectedTags.add(tagIds)
        detectTagSets.append(tagIds)
        usefulTags.append(tags)
print("DONE")
del manualControl, at_detector, robot
pygame.display.quit()
pygame.quit()

In [ ]:
len(usefulTags[0])

In [ ]:
def tagToT(tag):
    T = np.eye(4,dtype = float)
    T[:3,:3] = tag.pose_R
    T[:3, 3] = tag.pose_t.squeeze()
    return T

print(tagToT(usefulTags[0][1]))

In [ ]:
len(usefulTags)

In [ ]:
# 3 to 1
curr = usefulTags[1]
idxFrom = 1 # 3
idxTo = 0 # 1
print([tag.tag_id for tag in curr])
T3to1 = np.linalg.inv(tagToT(curr[idxTo])) @ tagToT(curr[idxFrom])
print(repr(T3to1))

In [ ]:
# 2 to 1
curr = usefulTags[2]
idxFrom = 1 # 2
idxTo = 0 # 1
print([tag.tag_id for tag in curr])
T2to1 = np.linalg.inv(tagToT(curr[idxTo])) @ tagToT(curr[idxFrom])
print(repr(T2to1))

In [ ]:
# 4 to 1
curr = usefulTags[0]
idxFrom = 1 # 2
idxTo = 0 # 1
print([tag.tag_id for tag in curr])
T4to3 = np.linalg.inv(tagToT(curr[idxTo])) @ tagToT(curr[idxFrom])
print(repr(T3to1 @ T4to3))

In [10]:
class Map:
  _landmarks = {
      1: np.array([[1, 0,0,0],
                      [0,0,1,0],
                      [0,-1,0,0],
                      [0,0,0,1]], dtype=float),
      3: np.array([[0.99471329, -0.01590299, -0.10145225,  1.04627609],
                   [0.02096435,  0.99857778,  0.0490196,  0.02472891],
                   [0.1005284, -0.05088733,  0.99363198,  0.02251224],
                   [0.,  0.,  0.,  1.]]),
      2: np.array([[-3.86255540e-02,  3.89327231e-02, -9.98495022e-01, -2.59944926e-01],
                   [7.12936093e-04,  9.99241520e-01,  3.89342511e-02, -2.82976469e-02],
                   [9.99253501e-01,  7.91993879e-04, -3.86240139e-02, -1.05946318e+00],
                   [0.00000000e+00,  0.00000000e+00,  0.00000000e+00, 1.00000000e+00]]),
      4: np.array([[-1.15711296e-02,  7.91264896e-04,  9.99932739e-01, 1.88734180e+00],
                   [-2.29588177e-02,  9.99735853e-01, -1.05678642e-03, 1.74137121e-03],
                   [-9.99669446e-01, -2.29695017e-02, -1.15499066e-02, -5.35296744e-01],
                   [0.00000000e+00,  0.00000000e+00,  0.00000000e+00, 1.00000000e+00]])
  }

  def getLandmark(id):
    if id == 1:
      return Map._landmarks[1]
    return Map._landmarks[1] @ Map._landmarks[id]

for id in range(1,5):
    print(id)
    print(Map.getLandmark(id))

1
[[ 1.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0. -1.  0.  0.]
 [ 0.  0.  0.  1.]]
2
[[-3.86255540e-02  3.89327231e-02 -9.98495022e-01 -2.59944926e-01]
 [ 9.99253501e-01  7.91993879e-04 -3.86240139e-02 -1.05946318e+00]
 [-7.12936093e-04 -9.99241520e-01 -3.89342511e-02  2.82976469e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
3
[[ 0.99471329 -0.01590299 -0.10145225  1.04627609]
 [ 0.1005284  -0.05088733  0.99363198  0.02251224]
 [-0.02096435 -0.99857778 -0.0490196  -0.02472891]
 [ 0.          0.          0.          1.        ]]
4
[[-1.15711296e-02  7.91264896e-04  9.99932739e-01  1.88734180e+00]
 [-9.99669446e-01 -2.29695017e-02 -1.15499066e-02 -5.35296744e-01]
 [ 2.29588177e-02 -9.99735853e-01  1.05678642e-03 -1.74137121e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
